In [1]:
import sys 
import os
import json
from pydantic.json import pydantic_encoder
import time
from dotenv import load_dotenv
load_dotenv('../../backend/.env')
sys.path.append("../../")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Reviewer import Reviewer
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import DraftResult


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
reviewer = Reviewer(ollama_client)


Performing health check on http://localhost:11434
Ollama is running
Health check passed


In [2]:
drafts_generated_path = "../one_vs_multi_shots/results"
path_to_emails = "../../dataset"
method = "single_shot"

already_evaluated = os.listdir("results")
    
for draft_file in os.listdir(drafts_generated_path):
    
    if draft_file in already_evaluated:
        print(f"Already evaluated {draft_file}")
        continue
    
    with open(f"{drafts_generated_path}/{draft_file}", 'r') as f:
        draft_generated_data = json.load(f)
        response_email = draft_generated_data[method]['response']['textResponse']
        sources = draft_generated_data[method]['response']['sources']    
        
    enquiry_path = f"{path_to_emails}/{draft_file.replace('results_', '')}"
    print(enquiry_path)
    with open(enquiry_path, 'r') as f:
        email_data = json.load(f)
        email = email_data['email']
    
    
    base_prompt = f"""
    From this enquiry:
    Enquiry: {email}
    ---
    Does the response answer the enquiry?
    Response: {response_email}
    ---
    """
    
    
    start_time_linkert = time.time()
    linkert_score = reviewer.linkert_eval(base_prompt)
    end_time_linkert = time.time()
    
    start_time_hallucination = time.time()
    draft_score = reviewer.hallucination_eval(base_prompt, sources)
    end_time_hallucination = time.time()
    
    results_path = f"results/reviewer_{draft_file}"
    
    with open(results_path, 'w') as f:
        json.dump({
            "linkert_score": linkert_score,
            "hallucination_score": draft_score,
            "linkert_time": end_time_linkert - start_time_linkert,
            "hallucination_time": end_time_hallucination - start_time_hallucination
        }, f)
        

../../dataset/fake_email_10.json
../../dataset/fake_email_undergrad_18.json
../../dataset/fake_email_undergrad_3.json
the JSON object must be str, bytes or bytearray, not NoneType
../../dataset/fake_email_3.json
../../dataset/fake_email_undergrad_14.json
../../dataset/fake_email_undergrad_15.json
../../dataset/fake_email_2.json
../../dataset/fake_email_undergrad_2.json
../../dataset/fake_email_undergrad_19.json
../../dataset/fake_email_11.json
../../dataset/fake_email_undergrad_12.json
../../dataset/fake_email_9.json
../../dataset/fake_email_16.json
../../dataset/fake_email_5.json
../../dataset/fake_email_4.json
../../dataset/fake_email_17.json
../../dataset/fake_email_8.json
../../dataset/fake_email_undergrad_13.json
../../dataset/fake_email_18.json
../../dataset/fake_email_7.json
../../dataset/fake_email_undergrad_7.json
the JSON object must be str, bytes or bytearray, not NoneType
../../dataset/fake_email_14.json
../../dataset/fake_email_15.json
../../dataset/fake_email_undergrad_6.

In [18]:
already_evaluated = os.listdir("results")
markdowns_path = "markdowns"
os.makedirs(markdowns_path, exist_ok=True)

for draft_file in os.listdir(drafts_generated_path):
    filename = f"reviewer_results_{draft_file}"
    if f"{filename}" not in already_evaluated:
        print(f"not evaluated {filename}")
        continue
    
    with open(f"{drafts_generated_path}/{draft_file}", 'r') as f:
        draft_generated_data = json.load(f)
        response_email = draft_generated_data[method]['response']['textResponse']
        sources = draft_generated_data[method]['response']['sources']    
        
    enquiry_path = f"{path_to_emails}/{draft_file.replace('results_', '')}"
    with open(enquiry_path, 'r') as f:
        email_data = json.load(f)
        email = email_data['email']
    
    with open(f"{markdowns_path}/human_reviewer_{draft_file.replace(".json", ".md")}", 'w') as f:
        # email and response in the markdown 
        text = f"""
# Enquiry {draft_file}

{email}

# Response

{response_email}

        """
        f.write(text)